### 2.1.1 启动线程

In [1]:
#include <iostream>
#include <thread>

In [2]:
namespace n2 {
    void do_something()
    {
        std::cout << "do_something" << "\n";
    }

    void do_something_else()
    {
        std::cout << "do_something_else" << "\n";
    }

    class background_task
    {
    public:
        void operator()() const
        {
            do_something();
            do_something_else();
        }
    };
}

In [3]:
{
    using namespace n2;
    background_task f;
    std::thread my_thread(f);

    my_thread.join();
}

do_something
do_something_else


有件事需要注意，当把函数对象传入到线程构造函数中时，需要避免“**最令人头痛的语法解析**”(C++’s most vexing parse, 中文简介)。如果你传递了一个临时变量，而不是一个命名的变量；C++编译器会将其解析为函数声明，而不是类型对象的定义。

In [4]:
/*
这里相当与声明了一个名为my_thread的函数，这个函数带有一个参数(函数指针指向没有参数并返回background_task对象的函数)，
返回一个 std::thread 对象的函数，而非启动了一个线程。
*/
{
    using namespace n2;
    std::thread my_thread(background_task());
}

input_line_6:8:26: warning: parentheses were disambiguated as a function declaration [-Wvexing-parse]
    std::thread my_thread(background_task());
                         ^~~~~~~~~~~~~~~~~~~
input_line_6:8:27: note: add a pair of parentheses to declare a variable
    std::thread my_thread(background_task());
                          ^
                          (                )


In [5]:
// 使用在前面命名函数对象的方式，或使用多组括号①，或使用新统一的初始化语法②，可以避免这个问题。
{
    using namespace n2;
    
    background_task f;
    std::thread my_thread0(f);

    std::thread my_thread1((background_task())); // 1
    std::thread my_thread2{background_task()}; // 2
    
    my_thread0.join();
    my_thread1.join();
    my_thread2.join();
}

do_something
do_something_else
do_something
do_something_else
do_something
do_something_else


In [6]:
// 使用lambda表达式也能避免这个问题
{
    using namespace n2;
    std::thread my_thread([]{
        do_something();
        do_something_else();
    });
    my_thread.join();
}

do_something
do_something_else


### 2.1.3 特殊情况下的等待
如果打算等待对应线程，则需要细心挑选调用join()的位置。当在线程运行之后产生**异常，在join()调用之前抛出，就意味着这次调用会被跳过**。

避免应用被抛出的异常所终止，就需要作出一个决定。通常，当倾向于在无异常的情况下使用join()时，需要在异常处理过程中调用join()，从而避免生命周期的问题。下面的程序清单是一个例子。

In [7]:
#include <iostream>
#include <thread>

In [8]:
namespace n8 {
    void do_something(unsigned i)
    {
        std::cout << "do_something: i = " << i << "\n";
    }

    struct func
    {
        int& i;
        func(int& i_) : i(i_) {}
        void operator() () {
            for (unsigned j = 0; j < 3; ++j) {
                do_something(i);
            }
        }
    };
}

In [9]:
// 代码使用了 try/catch 块确保访问本地状态的线程退出后，函数才结束。当函数正常退出时，会执行到2处；
// 当函数执行过程中抛出异常，程序会执行到1处。
{
    using namespace n8;

    int some_local_state = 0;
    func my_func(some_local_state);
    std::thread t(my_func);

    try {
        //do_something_in_current_thread();
    }
    catch(...) {
        t.join(); // 1
        throw;
    }
    t.join(); // 2
}

do_something: i = 0
do_something: i = 0
do_something: i = 0


一种方式是使用“资源获取即初始化方式”(RAII，Resource Acquisition Is Initialization)，并且
提供一个类，在析构函数中使用join()，如同下面清单中的代码。看它如何简化f()函数。

In [10]:
{
    using namespace n8;
    
    class thread_guard
    {
        std::thread& t;
    public:
        explicit thread_guard(std::thread& t_) : t(t_)
        {}
        ~thread_guard()
        {
            if(t.joinable()) // 1
            {
                t.join(); // 2
            }
        }
        thread_guard(thread_guard const&)=delete; // 3
        thread_guard& operator=(thread_guard const&)=delete;
    };
    
    int some_local_state=0;
    func my_func(some_local_state);
    std::thread t(my_func);
    thread_guard g(t);
    //do_something_in_current_thread();
}

do_something: i = 0
do_something: i = 0
do_something: i = 0


### 2.1.4 后台运行线程
调用 std::thread 成员函数detach()来分离一个线程。之后，相应的 std::thread 对象就与实际执行的线程无关了，并且这个线程也无法加入：

    std::thread t(do_background_work);
    t.detach();
    assert(!t.joinable());
    
为了从 std::thread 对象中分离线程(前提是有可进行分离的线程),不能对没有执行线程的 std::thread 对象**使用detach(),也是join()的使用条件**，并且要用同样的方式进行检查——当 std::thread 对象使用**t.joinable()返回的是true，就可以使用t.detach()**。